In [1]:
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting import Backtest
import pandas as pd
from backtesting.test import SMA
import pandas as pd
from bokeh.io import export_png
from backtesting.lib import SignalStrategy, TrailingStrategy
from backtesting import Position

/Users/fahad/anaconda3/lib/python3.7/site-packages/backtesting/_plotting.py:34: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `bokeh.io.reset_output()`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
MSFT_predictions = pd.read_csv("/Users/fahad/project_repo/models/final_models/Financial_Analysis/predictions/MSFT_predictions.csv", index_col=0)


In [3]:
MSFT_predictions.head()

Date  Predictions
0  2019-06-28            0
1  2019-05-31            0
2  2019-04-30            1
3  2019-03-29            1
4  2019-02-28            1

In [4]:
MSFT_predictions.Date = pd.to_datetime(MSFT_predictions.Date, format="%Y-%m-%d")

In [5]:
MSFT_predictions['Predictions'] = MSFT_predictions['Predictions'].shift(-1)


In [6]:
MSFT_predictions = MSFT_predictions.dropna()


In [7]:
MSFT_predictions = MSFT_predictions[::-1]

In [8]:
MSFT_predictions.head(10)

Date  Predictions
111 2010-03-31          1.0
110 2010-04-30          0.0
109 2010-05-28          0.0
108 2010-06-30          1.0
107 2010-07-30          0.0
106 2010-08-31          1.0
105 2010-09-30          1.0
104 2010-10-29          1.0
103 2010-11-30          1.0
102 2010-12-31          1.0

In [9]:
MSFT_monthly = pd.read_csv("/Users/fahad/project_repo/data/external/MSFT_monthly.csv", index_col =0)

In [10]:
MSFT_monthly.columns = ['Open','High','Low','Close','Volume']

In [11]:
MSFT_predictions = MSFT_predictions.set_index("Date")

In [12]:
MSFT_monthly.head()

Open     High     Low   Close     Volume
2019-07-26  136.63  141.675  134.67  141.34  412494800
2019-06-28  123.85  138.400  119.01  133.96  508324017
2019-05-31  130.53  130.650  123.04  123.68  547217986
2019-04-30  118.95  131.370  118.10  130.60  433157427
2019-03-29  112.89  120.820  108.80  117.94  589044900

In [13]:
MSFT_final_df = pd.merge(MSFT_predictions, MSFT_monthly, right_index=True, left_index=True)

In [14]:
MSFT_final_df.index = pd.to_datetime(MSFT_final_df.index, format ="%Y-%m-%d")

In [15]:
equity_dict = dict()

In [16]:
class Binary_Classification_1d(TrailingStrategy):

    
    def init(self):
        
        super().init()
        self.data.Predictions
        self.set_trailing_sl(4)
        
    def next(self):
        super().init()
        equity_dict[self.data.index[-1]] = TrailingStrategy.equity.__get__(self)
        
        position = Strategy.position.__get__(self)
        
        if self.data.Predictions[-1] == 1.0 and (position.is_long!=True) :
#             print("\n")
#             print (self.data.Prediction[-1])
            
            self.buy()

        # Else, if sma1 crosses below sma2, sell it
        elif self.data.Predictions[-1] == 0.0 and (position.is_short!=True)  :
#             print("\n")
#             print (self.data.Prediction[-1])
            
            self.sell()
            
 

In [17]:
bt = Backtest(MSFT_final_df, Binary_Classification_1d, cash=10000, commission=.01)
bt.run()



Start                          2010-03-31 00:00:00
End                            2019-06-28 00:00:00
Duration                        3376 days 00:00:00
Exposure [%]                               93.8981
Equity Final [$]                            247857
Equity Peak [$]                             247857
Return [%]                                 2378.57
Buy & Hold Return [%]                      357.397
Max. Drawdown [%]                         -12.3903
Avg. Drawdown [%]                         -4.22331
Max. Drawdown Duration           242 days 00:00:00
Avg. Drawdown Duration            90 days 00:00:00
# Trades                                        59
Win Rate [%]                               69.4915
Best Trade [%]                             24.6703
Worst Trade [%]                            -8.8236
Avg. Trade [%]                             4.97406
Max. Trade Duration              183 days 00:00:00
Avg. Trade Duration               54 days 00:00:00
Expectancy [%]                 

In [18]:
bt.plot(superimpose=False, plot_volume=False)

In [19]:
df = pd.DataFrame.from_dict([equity_dict])

In [20]:
df = df.T

In [21]:
df.columns = ['Equity']

In [22]:
export = df.to_csv("/Users/fahad/project_repo/data/final/Equity/CNN/MSFT_CNN_Equity.csv")

In [23]:
df

Equity
2010-04-30   10000.000000
2010-05-28   11473.139489
2010-06-30   12373.816449
2010-07-30   13665.923572
2010-08-31   14948.684816
2010-09-30   15203.326840
2010-10-29   16553.561052
2010-11-30   15679.788797
2010-12-31   17326.453740
2011-01-31   17211.606232
2011-02-28   17837.167115
2011-03-31   18568.601604
2011-04-29   18579.101606
2011-05-31   19069.355067
2011-06-30   18366.762528
2011-07-29   19267.810808
2011-08-31   19787.207602
2011-09-30   20977.782021
2011-10-31   22501.133245
2011-11-30   22420.569780
2011-12-30   22562.804978
2012-01-31   25665.625232
2012-02-29   27586.418722
2012-03-30   28034.024444
2012-04-30   27902.750287
2012-05-31   30333.753203
2012-06-29   32334.123657
2012-07-31   33248.913564
2012-08-31   34158.615038
2012-09-28   32983.789213
...                   ...
2017-01-31  104003.660714
2017-02-28  103101.344758
2017-03-31  104591.375903
2017-04-28   99310.438013
2017-05-31   99639.625153
2017-06-30   98341.342523
2017-07-31  103719.941990
2017-08-31  106673.178303
2017-09-29  106273.706724
2017-10-31  118671.592500
2017-11-30  117510.751824
2017-12-29  119862.889784
2018-01-31  133132.723386
2018-02-28  131395.173896
2018-03-29  134173.116521
2018-04-30  138459.198959
2018-05-31  146335.620457
2018-06-29  145995.098474
2018-07-31  157054.660239
2018-08-31  146191.378687
2018-09-28  151556.447745
2018-10-31  160971.907002
2018-11-30  164772.241660
2018-12-31  183060.816042
2019-01-31  193219.861480
2019-02-28  207281.634412
2019-03-29  218216.513099
2019-04-30  241640.466430
2019-05-31  228836.852129
2019-06-28  247857.250253

[111 rows x 1 columns]